In [ ]:
#load libraries
!pip install beautifulsoup4
!pip install lxml
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
print("Libraries Imported.")

     |████████████████████████████████| 112kB 6.0MB/s eta 0:00:01


In [ ]:
#PART 1 of Project
#parse data from wikipedia with Beautiful Soup and Python
req = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695")
soup = BeautifulSoup(req.content,'html.parser')
table = soup.find_all('table')

#create dataframe from parsed data
df = pd.read_html(str(table))[0]
df

In [ ]:
#remove all 'not assigned' boroughs
z=df.loc[(df['Borough'] != "Not assigned")] 
print(z)

In [ ]:
#group by Postal Code and Borough.  Use lambda to add a comma to the Boroughs
grouped=z.groupby(['Postcode','Borough'], as_index=False).agg(lambda x: ', '.join(x))
grouped

In [ ]:
#Replace Not assigned data
grouped['Neighbourhood']=grouped['Neighbourhood'].replace('Not assigned', grouped['Borough'])
grouped.style.hide_index()

In [ ]:
#Get shape of dataframe
grouped.shape 

In [ ]:
#PART 2 of Project
#Get the latitude and the longitude coordinates of each neighborhood.
#Failed to load.  Read CSV to get coords.

!pip install geocoder
import geocoder
geo=pd.read_csv('Geospatial_Coordinates.csv')
geo

In [ ]:
#Renamed grouped dataframe to make merger of frames easier
grouped.rename(columns={"Postcode" : "Postal Code"}, inplace=True)
grouped

In [ ]:
#Merged frames 'grouped' and 'geo'
merged_grouped = pd.merge(grouped, geo, on='Postal Code')
merged_grouped.head(20)

In [ ]:
#Import required libraries for mapping, plotting and clustering

import folium

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!pip install geopy
from geopy.geocoders import Nominatim

import requests #
from pandas.io.json import json_normalize 

print('Libraries imported.')

In [ ]:
#Link to Foursquare
CLIENT_ID = 'FOONXVWYCI4P53VX4WN5H3GWAMFB5JIZBD0KLUI1AI0UFCLL' # your Foursquare ID
CLIENT_SECRET = 'OTN4Y4O2HQVT3OZSR3W5CV3QKFFY00OCPEDKSW1WHLWUWHTJ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
RADIUS = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
#Find Toronto using 
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Toronto using Folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [ ]:
#Frame for data
merged_grouped.head(20)

In [ ]:
#Create Toronto Map with Markers

for lat, lng, borough, neighborhood in zip(
        merged_grouped['Latitude'], 
        merged_grouped['Longitude'], 
        merged_grouped['Borough'], 
        merged_grouped['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [ ]:
#Explore boroughs that contain the word Toronto ONLY
toronto_table=merged_grouped[merged_grouped['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_table

In [ ]:
#Update Map for 4 Neighbourhoods ONLY
for lat, lng, borough, neighborhood in zip(
        toronto_table['Latitude'], 
        toronto_table['Longitude'], 
        toronto_table['Borough'], 
        toronto_table['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [ ]:
#Find the 1st Neighbourhood in Toronto
main_neighbourhood = toronto_table.loc[0, 'Neighbourhood']
main_neighbourhood

In [ ]:
#Use Foursquare API to search The Beaches and get JSON file
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    43.68, 
   -79.29, 
    RADIUS, 
    LIMIT)
results = requests.get(url).json()

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
#Normailize the file and create a new dataframe
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues)
nearby_venues


# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

In [ ]:
#Create a Venues Map
venues_map = folium.Map(location=[43.68,-79.29], zoom_start=15)

# add a blue circle marker to represent The Beaches
folium.features.CircleMarker(
    [43.68, -79.29],
    radius=10,
    color='blue',
    popup='The Beaches',
    fill = True,
    fill_color = 'blue',
    fill_opacity = 0.6
).add_to(venues_map)

# add all venues as red circle markers
for lat, lng, label in zip(nearby_venues.lat, nearby_venues.lng, nearby_venues.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)
venues_map